In [1]:
import matplotlib

import matplotlib.font_manager as fm

font_location = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

font_name = fm.FontProperties(fname = font_location).get_name()

matplotlib.rc('font',family = font_name)

In [7]:
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix

In [3]:
# 데이터 불러오기
df_raw=pd.read_csv("유방암.csv", engine="python")
df_raw.head()

,diagnosis,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
# 목표변수, 설명변수 데이터 지정
df_raw_y=df_raw["diagnosis"]
df_raw_x=df_raw.drop("diagnosis", axis=1, inplace=False)

#train_test_split(x: 설명변수 데이터, y: 목표변수 데이터, test_size=test 데이터 비율, random_state: random seed)
df_train_x, df_test_x, df_train_y, df_test_y=train_test_split(df_raw_x, df_raw_y, test_size=0.3, random_state=1234)
print("train data X size : {}".format(df_train_x.shape))
print("train data Y size : {}".format(df_train_y.shape))
print("test data X size : {}".format(df_test_x.shape))
print("test data Y size : {}".format(df_test_y.shape))

train data X size : (398, 30)
train data Y size : (398,)
test data X size : (171, 30)
test data Y size : (171,)


In [5]:
# train은 0.4, train의 validation은 0.3이 되도록 train_data를 다시 분할
df_train_x, df_val_x, df_train_y, df_val_y=train_test_split(df_train_x, df_train_y, test_size=3/7)

print("train data X size : {}".format(df_train_x.shape))
print("train data Y size : {}".format(df_train_y.shape))
print("validation data X size : {}".format(df_val_x.shape))
print("validation data Y size : {}".format(df_val_y.shape))

train data X size : (227, 30)
train data Y size : (227,)
validation data X size : (171, 30)
validation data Y size : (171,)


In [24]:
# 기본 옵션으로 모델 생성
# knn 모델 생성
knn_uncustomized=KNeighborsClassifier()
knn_uncustomized.fit(df_train_x, df_train_y)

# train 데이터 셋 정확도
print("Accuracy on training set: {:.3f}".format(knn_uncustomized.score(df_train_x, df_train_y)))

# validation set 정확도
print("Accuracy on validation set: {:.3f}".format(knn_uncustomized.score(df_val_x, df_val_y)))

# test accuracy
print("Accuracy on test set:{:.3f}".format(knn_uncustomized.score(df_test_x, df_val_y)))

Accuracy on training set: 0.943
Accuracy on validation set: 0.906
Accuracy on test set:0.503


In [9]:
knn_uncustomized

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

#### 모델 정확도는 train 94.3%, validation 90.6%로 성능은 유사한 수준이나 다소 과대적합이다.

In [11]:
# 모델 생성
# 이웃 수 증가에 따른 정확도 변화

# train 및 test 정확도 결과 저장용
train_accuracy=[]
validation_accuracy=[]

# 최대 이웃 수: 1-15
para_n_neighbors=[i for i in range(1,16)]


for v_n_neighbors in para_n_neighbors:
    knn=KNeighborsClassifier(n_neighbors=v_n_neighbors)
    knn.fit(df_train_x, df_train_y)
    train_accuracy.append(knn.score(df_train_x, df_train_y))
    validation_accuracy.append(knn.score(df_val_x, df_val_y))
    
# 데이터 테이블로 저장
df_accuracy_neighbors=pd.DataFrame()
df_accuracy_neighbors["Neighbors"]=para_n_neighbors
df_accuracy_neighbors["TrainAccuracy"]=train_accuracy
df_accuracy_neighbors["ValidationAccuracy"]=validation_accuracy


df_accuracy_neighbors.round(3)

,Neighbors,TrainAccuracy,ValidationAccuracy
0,1,1.000,0.901
1,2,0.960,0.901
2,3,0.960,0.912
3,4,0.938,0.906
4,5,0.943,0.906
5,6,0.934,0.912
6,7,0.952,0.918
7,8,0.943,0.918
8,9,0.952,0.918
9,10,0.938,0.924


#### 군집이 10일 때 validation data의 정확도가 0.924로 가장 높고, 같은 정확도에서 train accuracy의 정확도도 가장 높다.
#### n_neighbors=10으로 선택

In [14]:
# 가중치별 이웃 수 증가에 따른 정확도 변화

# train 및 test 정확도 결과 저장용
train_accuracy=[]
validation_accuracy=[]

# 이웃 수
para_n_neighbors=[i for i in range(1,11)]*2

# 가중치
para_weights=(["uniform"])*10+(["distance"]*10)

for (v_n_neighbors, v_weights) in zip(para_n_neighbors, para_weights):
    knn=KNeighborsClassifier(n_neighbors=v_n_neighbors, weights=v_weights)
    knn.fit(df_train_x, df_train_y)
    train_accuracy.append(knn.score(df_train_x, df_train_y))
    validation_accuracy.append(knn.score(df_val_x, df_val_y))
    
# 데이터 테이블로 저장
df_accuracy_weights=pd.DataFrame()
df_accuracy_weights["Neighbors"]=para_n_neighbors
df_accuracy_weights["Weights"]=para_weights
df_accuracy_weights["TrainAccuracy"]=train_accuracy
df_accuracy_weights["ValidationAccuracy"]=validation_accuracy
df_accuracy_weights.round(3)

,Neighbors,Weights,TrainAccuracy,ValidationAccuracy
0,1,uniform,1.000,0.901
1,2,uniform,0.960,0.901
2,3,uniform,0.960,0.912
3,4,uniform,0.938,0.906
4,5,uniform,0.943,0.906
5,6,uniform,0.934,0.912
6,7,uniform,0.952,0.918
7,8,uniform,0.943,0.918
8,9,uniform,0.952,0.918
9,10,uniform,0.938,0.924


#### 가중치를 uniform으로 할 때 neighbors가 10일 때 validation accuracy가 0.924로 가장 높음
#### 가중치를 distance로 할 때 neighbors가 5일 때 validation accuracy가 0.924로 가장 높지만 train accuracy가 1로 과대적합이므로 사용할 수 없음
#### uniform 가중치로 10 neighbors 선택

In [17]:
# 거리 계산 방법별 이웃 수 증가에 따른 정확도 변화

# train 및 test 정확도 결과 저장용
train_accuracy=[]
validation_accuracy=[]

# 이웃 수
para_n_neighbors=[i for i in range(1,11)]*3
# 거리 계산 방법
para_metric=["minkowski"]*10+["euclidean"]*10+["manhattan"]*10

for (v_n_neighbors, v_metric) in zip(para_n_neighbors, para_metric):
    knn=KNeighborsClassifier(n_neighbors=v_n_neighbors, metric=v_metric, weights="uniform")
    knn.fit(df_train_x, df_train_y)
    train_accuracy.append(knn.score(df_train_x, df_train_y))
    validation_accuracy.append(knn.score(df_val_x, df_val_y))
    
# 데이터 테이블로 저장
df_accuracy_metric=pd.DataFrame()
df_accuracy_metric["Neighbors"]=para_n_neighbors
df_accuracy_metric["Metric"]=para_metric
df_accuracy_metric["TrainAccuracy"]=train_accuracy
df_accuracy_metric["ValidationAccuracy"]=validation_accuracy
df_accuracy_metric.round(3)

,Neighbors,Metric,TrainAccuracy,ValidationAccuracy
0,1,minkowski,1.000,0.901
1,2,minkowski,0.960,0.901
2,3,minkowski,0.960,0.912
3,4,minkowski,0.938,0.906
4,5,minkowski,0.943,0.906
5,6,minkowski,0.934,0.912
6,7,minkowski,0.952,0.918
7,8,minkowski,0.943,0.918
8,9,minkowski,0.952,0.918
9,10,minkowski,0.938,0.924


#### 거리 계산 방법을 minkowski로 할 때 neighbors가 10일 때 validation accuracy가 0.924로 가장 높음
#### 거리 계산 방법을 euclidean으로 할 때 neighbors가 10일 때 validation accuracy가 0.924로 가장 높음
#### 거리 계산 방법을 manhattan으로 할 때 neighbors가 8일 때 validation accuracy가 0.924로 가장 높음
#### train data가 덜 fitting되고 validation accuracy가 가장 높은 거리 계산 방법=euclidean, neighbors=10으로 선택

In [23]:
# 결론 도출
# 최종 모델
knn_model=KNeighborsClassifier(n_neighbors=4, weights="uniform", metric="euclidean")
knn_model.fit(df_train_x, df_train_y)

# 평가
y_pred=knn_model.predict(df_test_x)


# train 데이터 셋 정확도
print("Accuracy on training set: {:.3f}".format(knn_model.score(df_train_x, df_train_y)))
# test 데이터 셋 정확도
print("Accuracy on test set: {:.3f}\n".format(knn_model.score(df_test_x, df_test_y)))
# confusion matrix
print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))

# precision 
print("Precision: {:.3f}".format(104/(104+12)))

Accuracy on training set: 0.938
Accuracy on test set: 0.924

Confusion matrix: 
[[104   1]
 [ 12  54]]
Precision: 0.897


#### test accuracy는 92.4%로 기본 옵션의 모형의 50.3%보다 높다. 
#### train accuray는 93.8%로 test accuracy보다 약 1.4% 높다.
#### Confusion matrix로부터 구한 유방암 환자의 정밀도(precision)은 89.7%이다. 즉, 암 환자가 암환자임을 정확하게 밝혀낼 수 있는 확률은 89.7%이다.
#### 서포트 벡터 머신 모형, 인공신경망 모형, knn 모형 중 가장 좋은 모형은 정확도와 정밀도를 고려할 때 knn 모형이라고 할 수 있다.